<a href="https://colab.research.google.com/github/kylematoba/deeplearning-project/blob/master/autoaugment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import logging
import sys
import pprint

import tensorflow as tf
# formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

# format = '%(asctime)s %(levelname)-8s %(message)s'
# screen_handler = logging.StreamHandler(stream=sys.stdout)
# logging.basicConfig(level=logging.DEBUG, format=format)

FORMAT = "%(asctime)s %(process)s %(thread)s: %(message)s"
logging.basicConfig(level=logging.DEBUG, format=FORMAT, stream=sys.stdout)
logger = logging.getLogger(__name__)
      
# logger = logging.getLogger(__name__)
# logger.setLevel(logging.DEBUG)
# logger.addHandler(screen_handler)
  
logger.info(sys.version)
logger.info("{}".format("Running in ipython" if 'ipykernel' in sys.modules else ""))
logger.info("TensorFlow version {}".format(tf.__version__))

# https://stackoverflow.com/questions/49322072/checkpoints-in-google-colab

2018-11-14 07:25:42,387 65 139889902393216: 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
2018-11-14 07:25:42,388 65 139889902393216: Running in ipython
2018-11-14 07:25:42,390 65 139889902393216: TensorFlow version 1.12.0


In [2]:
envir_str = pprint.pformat(dict(os.environ), indent=4)
log_envir = True
if log_envir:
    logger.info(envir_str)

2018-11-14 07:25:48,156 65 139889902393216: {   'CLICOLOR': '1',
    'CLOUDSDK_CONFIG': '/content/.config',
    'COLAB_GPU': '1',
    'CUDA_PKG_VERSION': '9-2=9.2.148-1',
    'CUDA_VERSION': '9.2.148',
    'CUDNN_VERSION': '7.3.1.20',
    'DATALAB_SETTINGS_OVERRIDES': '{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["notebook","-y","--no-browser","--log-level=DEBUG","--debug","--NotebookApp.allow_origin=\\"*\\"","--NotebookApp.log_format=\\"%(message)s\\"","--NotebookApp.disable_check_xsrf=True","--NotebookApp.token=","--Session.key=\\"\\"","--Session.keyfile=\\"\\"","--ContentsManager.untitled_directory=\\"Untitled '
                                  'Folder\\"","--ContentsManager.untitled_file=\\"Untitled '
                                  'File\\"","--ContentsManager.untitled_notebook=\\"Untitled '
                                  'Notebook\\"","--KernelManager.autorestart=True","--ip=\\"172.28.0.2\\""]}',
    'DEBIAN_FRONTEND': 'noninteractive'

In [3]:
from google.colab import drive
mount_loc = '/content/gdrive'
force_remount = True
drive.mount(mount_loc, force_remount=force_remount)

# drive.mount("/content/gdrive", force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)
# sess

# tf.GPUOptions(per_process_gpu_fraction=1)
tf.GPUOptions(per_process_gpu_memory_fraction=1.0)

per_process_gpu_memory_fraction: 1.0

In [0]:
my_drive = os.path.join(mount_loc, 'My\ Drive')
local_home = "./local_home"
os.makedirs(local_home, exist_ok=True)
# symlink_name = os.path.join(mount_loc, 'my_drive')
symlink_name = local_home
!ln -s $my_drive $symlink_name
!mv $symlink_name/'My Drive' $symlink_name/my_drive

In [6]:
!ls -lart local_home/my_drive | head
# !ls my_drive/'My Drive'
# os.listdir('my_drive/My\ Drive')
# !ls $symlink_name/my_drive

lrwxrwxrwx 1 root root 24 Nov 14 07:26 local_home/my_drive -> /content/gdrive/My Drive


In [0]:
do_cifar10_download = True
if do_cifar10_download:
  logger.info('Downloading CIFAR data')
  !curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
  !tar -xvzf cifar-10-python.tar.gz

2018-11-14 07:26:54,752 65 139889902393216: Downloading CIFAR data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 23  162M   23 37.4M    0     0  1682k      0  0:01:38  0:00:22  0:01:16 3324k

In [0]:
remove_clone_if_present = True
if remove_clone_if_present:
    !rm -rf models

In [9]:
do_clone = True

if do_clone and (not os.path.isdir('models')):
    logger.info('Cloning repo')
    !git clone https://github.com/kylematoba/models.git
    !cd models
!git --git-dir=./models/.git log --name-status HEAD^..HEAD

2018-11-13 22:18:13,402 63 140539550672768: Cloning repo
Cloning into 'models'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22399 (delta 6), reused 10 (delta 3), pack-reused 22377
Receiving objects: 100% (22399/22399), 559.88 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (13235/13235), done.
Checking out files: 100% (2837/2837), done.
commit 83415dfdda6b9755c587e557f6cee6f5640b17cd (HEAD -> master, origin/master, origin/HEAD)
Author: Kyle Matoba <kylematoba@gmail.com>
Date:   Tue Nov 13 22:02:35 2018 +0000

    refactoring

M	research/autoaugment/augmentation_transforms.py
M	research/autoaugment/data_utils.py
M	research/autoaugment/helper_utils.py
M	research/autoaugment/load_cifar101.py
M	research/autoaugment/policies.py
M	research/autoaugment/shake_shake.py
D	research/autoaugment/shakespeare.py
A	research/autoaugment/tpu_mnist_example.py
M	research/autoaugment/train_cifar.py


In [0]:
mount_loc = './local_home/my_drive'
# model_name = 'wrn'

# model_name = 'pyramid_net'
# model_name = 'shake_shake_112'
# model_name = 'shake_shake_32'
model_name = 'shake_shake_96'
# base_dir = os.path.join(mount_loc, 'My Drive')
base_dir = mount_loc
checkpoint_dir = os.path.join(base_dir, model_name)
os.makedirs(checkpoint_dir, exist_ok=True)
# use_cpu = 1
use_cpu = 0

In [0]:
logger.info("Running {}".format(model_name))
logger.info("Checkpoint dir {}".format(checkpoint_dir))
# https://ipython.readthedocs.io/en/stable/interactive/python-ipython-diff.html
# !python3 models/research/autoaugment/train_cifar.py --model_name=shake_shake_32 --checkpoint_dir=./shake_shake_32 --data_path=cifar-10-batches-py --dataset='cifar10' --use_cpu=0
!python3 models/research/autoaugment/train_cifar.py --model_name=$model_name --checkpoint_dir=$checkpoint_dir --data_path=cifar-10-batches-py --dataset='cifar10' --use_cpu=$use_cpu
# !python3 models/research/autoaugment/train_cifar.py --model_name=$model_name --checkpoint_dir=local_home --data_path=cifar-10-batches-py --dataset='cifar10' --use_cpu=0

2018-11-13 22:19:50,471 63 140539550672768: Running shake_shake_96
2018-11-13 22:19:50,474 63 140539550672768: Checkpoint dir ./local_home/my_drive/shake_shake_96
INFO:tensorflow:Cifar10
INFO:tensorflow:loading file: cifar-10-batches-py/data_batch_1
INFO:tensorflow:loading file: cifar-10-batches-py/data_batch_2
INFO:tensorflow:loading file: cifar-10-batches-py/data_batch_3
INFO:tensorflow:loading file: cifar-10-batches-py/data_batch_4
INFO:tensorflow:loading file: cifar-10-batches-py/data_batch_5
INFO:tensorflow:loading file: cifar-10-batches-py/test_batch
tcmalloc: large alloc 1474560000 bytes == 0x25b2a000 @  0x7f07bc5861e7 0x7f07b9e95a41 0x7f07b9ef8c13 0x7f07b9ef8cda 0x7f07b9fa64ed 0x7f07b9fa6e3b 0x7f07b9fa96c8 0x7f07b49c4287 0x7f07b49c51ef 0x59e27b 0x7f07b9fb1ae7 0x59aa00 0x4f8068 0x510375 0x5917a1 0x574e90 0x5264b1 0x59f9fc 0x511b0a 0x4f5277 0x510375 0x5917a1 0x574e90 0x5264b1 0x59f9fc 0x511b0a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x506ff4
INFO:tensorflow:mean: [0.49139968, 0.48215

In [12]:

# !ls -lrta local_home/my_drive/

# files.download('./model.ckpt-170.data-00000-of-00001')
!pip install dropbox
# !ls

    100% |████████████████████████████████| 501kB 6.5MB/s 


In [0]:
import os
import dropbox


access_token = '414hB1PEYcAAAAAAAAAACiJZR19B2H0RkPp7x8aqkrTKpjro91qJCIk8Sl8wK1Tz'


def _upload_big_file(file_path: str,
                     dest_path: str) -> None:
    f = open(file_path, 'rb')
    file_size = os.path.getsize(file_path)

    CHUNK_SIZE = 4 * 1024 * 1024

    if file_size <= CHUNK_SIZE:
        dbx.files_upload(f.read(), dest_path)
    else:
        upload_session_start_result = dbx.files_upload_session_start(f.read(CHUNK_SIZE))
        session_id = upload_session_start_result.session_id
        cursor = dropbox.files.UploadSessionCursor(session_id=session_id,
                                                   offset=f.tell())
        commit = dropbox.files.CommitInfo(path=dest_path)

        while f.tell() < file_size:
            if file_size - f.tell() <= CHUNK_SIZE:
                print(dbx.files_upload_session_finish(f.read(CHUNK_SIZE),
                                                cursor,
                                                commit))
            else:
                dbx.files_upload_session_append(f.read(CHUNK_SIZE),
                                                cursor.session_id,
                                                cursor.offset)
                cursor.offset = f.tell()


In [14]:
 !ls local_home/my_drive/$model_name/model

checkpoint			    model.ckpt-44.data-00000-of-00001
model.ckpt-136.data-00000-of-00001  model.ckpt-44.index
model.ckpt-136.index		    model.ckpt-44.meta
model.ckpt-136.meta		    model.ckpt-71.data-00000-of-00001
model.ckpt-160.data-00000-of-00001  model.ckpt-71.index
model.ckpt-160.index		    model.ckpt-71.meta
model.ckpt-160.meta		    model.ckpt-97.data-00000-of-00001
model.ckpt-161.index		    model.ckpt-97.index
model.ckpt-161.meta		    model.ckpt-97.meta


In [15]:
from typing import Tuple

def _get_num_from_model_name(model_name: str) -> int:
  if ('shake_shake_32' == model_name) or ('shake_shake_96' == model_name):
    num = 1799
  elif ('wrn' == model_name):
    num = 179
  else:
    raise ValueError('need to set up this model')
  return num


def _get_full_file_names(model_name: str, 
                         filename: str) -> Tuple[str, str]:
  dest_path = '/{}/{}'.format(model_name, filename)
  file_path = 'local_home/my_drive/{}/model/{}'.format(model_name, filename)
  return dest_path, file_path


def _upload_file_to_dropbox(model_name: str, 
                            filename: str) -> None:
  dest_path, file_path = _get_full_file_names(model_name, filename)
  logger.info('Writing {} to {}'.format(file_path, dest_path))
  _upload_big_file(file_path, dest_path)
  
  
def _upload_model_results(model_name: str) -> None:
  num = _get_num_from_model_name(model_name)
  filename = "model.ckpt-{}.data-00000-of-00001".format(num)
  _upload_file_to_dropbox(model_name, filename)

  filename = "checkpoint"
  _upload_file_to_dropbox(model_name, filename)
  
  filename = "model.ckpt-{}.index".format(num)
  _upload_file_to_dropbox(model_name, filename)
  
  filename = "model.ckpt-{}.meta".format(num)
  _upload_file_to_dropbox(model_name, filename)
  
  
  
_upload_model_results(model_name)

2018-11-13 18:57:37,422 60 139858692757376: Writing local_home/my_drive/shake_shake_96/model/model.ckpt-1799.data-00000-of-00001 to /shake_shake_96/model.ckpt-1799.data-00000-of-00001


FileNotFoundError: ignored

In [0]:
# !cp local_home/my_drive/wrn/model/model.ckpt-170.data-00000-of-00001 local_home/my_drive/

# files.download('local_home/my_drive/wrn/model/checkpoint')
# files.download('local_home/my_drive/wrn/model/model.ckpt-170.data-00000-of-00001')
# files.download('local_home/my_drive/wrn/model/model.ckpt-170.index')
# files.download('local_home/my_drive/wrn/model/model.ckpt-170.meta')

# dbx = dropbox.Dropbox(access_token)
# response = dbx.files_list_folder("")
# write_mode = dropbox.files.WriteMode('overwrite')

# _upload_big_file(file_path, dest_path)
# with open(file_path, 'rb') as f:
#     dbx.files_upload(f.read(),
#                  dest_path,
#                  mode=write_mode,
#                  mute=True)